In [88]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd

# Pho's Formatting Preferences
# from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
# set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.general_helpers import CodeConversion
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.print_helpers import print_keys_if_possible, print_value_overview_only, document_active_variables, objsize, print_object_memory_usage, debug_dump_object_member_shapes, TypePrintMode
from pyphocorehelpers.print_helpers import get_now_day_str, get_now_time_str, get_now_time_precise_str
from pyphocorehelpers.programming_helpers import IPythonHelpers
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot
from pyphoplacecellanalysis.General.Batch.NonInteractiveWrapper import batch_load_session, batch_extended_computations, SessionBatchProgress, batch_programmatic_figures, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap

# from PendingNotebookCode import _perform_batch_plot, _build_batch_plot_kwargs

session_batch_status = {}
session_batch_errors = {}
enable_saving_to_disk = False

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Pipeline

In [ ]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'

## Data must be pre-processed using the MATLAB script located here: 
#     neuropy/data_session_pre_processing_scripts/KDIBA/IIDataMat_Export_ToPython_2022_08_01.m
# From pre-computed .mat files:

local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=self.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

# # Animal `gor01`:
local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='gor01', exper_name='one') # IdentifyingContext<('kdiba', 'gor01', 'one')>
local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name) # 'gor01', 'one'
local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=['PhoHelpers', 'Spike3D-Minimal-Test', 'Unused'])

# local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='gor01', exper_name='two')
# local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name)
# local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=[])

# ## Animal `vvp01`:
# local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='vvp01', exper_name='one')
# local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name)
# local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=[])

# local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='vvp01', exper_name='two')
# local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name)
# local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=[])

# ### Animal `pin01`:
# local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='pin01', exper_name='one')
# local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name) # no exper_name ('one' or 'two') folders for this animal.
# local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=['redundant','showclus','sleep','tmaze'])

## Build session contexts list:
local_session_contexts_list = [local_session_parent_context.adding_context(collision_prefix='sess', session_name=a_name) for a_name in local_session_names_list] # [IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-07_11-26-53')>, ..., IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-13_14-42-6')>]

## Initialize `session_batch_status` with the NOT_STARTED status if it doesn't already have a different status
for curr_session_basedir in local_session_paths_list:
    curr_session_status = session_batch_status.get(curr_session_basedir, None)
    if curr_session_status is None:
        session_batch_status[curr_session_basedir] = SessionBatchProgress.NOT_STARTED # set to not started if not present
        # session_batch_status[curr_session_basedir] = SessionBatchProgress.COMPLETED # set to not started if not present

session_batch_status

In [ ]:
%pdb off
# %%viztracer
basedir = local_session_paths_list[1] # NOT 3
print(f'basedir: {str(basedir)}')

# Read if possible:
saving_mode = PipelineSavingScheme.SKIP_SAVING
force_reload = False

# # Force write:
# saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
# force_reload = True

# ==================================================================================================================== #
# Load Pipeline                                                                                                        #
# ==================================================================================================================== #
# epoch_name_whitelist = ['maze']
epoch_name_whitelist = None
active_computation_functions_name_whitelist=['_perform_baseline_placefield_computation', '_perform_time_dependent_placefield_computation', '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        # '_perform_pf_find_ratemap_peaks_computation',
                                        '_perform_time_dependent_pf_sequential_surprise_computation'
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding'
                                    ]
curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_whitelist=epoch_name_whitelist,
                                          computation_functions_name_whitelist=active_computation_functions_name_whitelist,
                                          saving_mode=saving_mode, force_reload=force_reload,
                                          skip_extended_batch_computations=True, debug_print=False, fail_on_exception=True)

In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path
# 2023-01-* - Call extended computations to build `_display_short_long_firing_rate_index_comparison` figures:
extended_computations_include_whitelist=['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses'] # do only specifiedl
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_whitelist=extended_computations_include_whitelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=True, debug_print=False)
newly_computed_values

In [ ]:
_perform_long_short_decoding_analyses

In [ ]:
## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
## Extract variables from results object:
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global

# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333

In [ ]:
global_session.replay

In [ ]:
long_results_obj.active_filter_epochs

In [ ]:
global_session.get_output_path()

In [ ]:
long_results_obj.active_filter_epochs

In [151]:
curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis'].long_mean_replays_frs
curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis'].short_mean_replays_frs


# curr_active_pipeline.global_computation_results.computed_data.long_short_fr_indicies_analysis


{2: 0.0,
 3: 3.5240238671668336,
 4: 5.45636121658812,
 5: 5.612399201388204,
 6: 56.242654480413115,
 7: 2.0891659488791414,
 8: 1.0663324739621252,
 9: 11.591117925119503,
 10: 9.896100987923246,
 11: 2.961471296531577,
 12: 102.5269567711694,
 13: 1.6180856129932144,
 14: 6.900302827342532,
 15: 4.45719684570725,
 16: 2.166239048419014,
 17: 0.5167876888228234,
 18: 7.420919269826718,
 19: 1.8149142336719806,
 20: 43.049752471241675,
 21: 1.4107566102872957,
 22: 2.2670341607590196,
 23: 0.49940179884005514,
 24: 0.0,
 25: 2.202361831386971,
 26: 0.2890938664448221,
 27: 1.447337533220852,
 28: 0.0,
 29: 39.51293108308353,
 30: 5.486710677199816,
 31: 9.397785061471893,
 32: 3.939118143167563,
 33: 4.117001318119649,
 34: 0.6421732024528712,
 35: 3.6279766087658305,
 36: 8.60579876535322,
 37: 4.357272903758823,
 38: 4.940047279359042,
 39: 9.386430601556238,
 40: 44.2032665997469,
 41: 5.514012694353516,
 42: 31.941533145335235,
 43: 4.665856064751075,
 44: 5.825912006545132,
 45: 

In [153]:


# Add Right-click > 'Insert reference' in the "WATCH" panel of "RUN AND DEBUG" to insert a reference to that variable in the notebook. Could also have a "Copy reference".
	# Currently has a "Copy Value" option that is close but only copies the value.
	# DISCOVERY: It already exists under the normal "VARIABLES" section, and is called "Copy as Expression"!!

rr_aclus = np.array(list(curr_active_pipeline.global_computation_results.computed_data.long_short_fr_indicies_analysis.y_frs_index.keys()))
rr_neuron_type = np.array([global_session.neurons.aclu_to_neuron_type_map[aclu] for aclu in rr_aclus])
rr_laps = np.array(list(curr_active_pipeline.global_computation_results.computed_data.long_short_fr_indicies_analysis.y_frs_index.values()))
rr_replays = np.array(list(curr_active_pipeline.global_computation_results.computed_data.long_short_fr_indicies_analysis.x_frs_index.values()))
rr_skew = rr_laps / rr_replays

## Sort on 'rr_replays'
sort_indicies = np.argsort(rr_replays)

## Sort all the variables:
rr_aclus = rr_aclus[sort_indicies]
rr_neuron_type = rr_neuron_type[sort_indicies]
rr_laps = rr_laps[sort_indicies]
rr_replays = rr_replays[sort_indicies]
rr_skew = rr_skew[sort_indicies]

# Build dataframe:
rate_remapping_df = pd.DataFrame({'aclu': rr_aclus, 'neuron_type': rr_neuron_type, 'laps': rr_laps, 'replays': rr_replays, 'skew': rr_skew})
rate_remapping_df.set_index('aclu', inplace=True)
rate_remapping_df.dropna('index', subset=['laps','replays'], how='any', inplace=True) # drop any cells that don't have any replays at all.

## Add distances from the center to indicate how much remapping that cell engaged in so we can find the cells that remapped the most:
rate_remapping_df['max_axis_distance_from_center'] =  np.max(np.vstack((np.abs(rate_remapping_df['laps'].to_numpy()), np.abs(rate_remapping_df['replays'].to_numpy()))), axis=0)
rate_remapping_df['distance_from_center'] = np.sqrt(rate_remapping_df['laps'].to_numpy() ** 2 + rate_remapping_df['replays'].to_numpy() ** 2)


rate_remapping_df
# rate_remapping_df.to_clipboard(index=False)

# rate_remapping_df.to_csv(f'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/rate_remapping_df.csv', index=False)

# rate_remapping_df = pd.read_csv(f'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/rate_remapping_df.csv')
# rate_remapping_df

In [180]:
## Find only the cells that exhibit considerable remapping:
considerable_remapping_threshold = 0.7 # cells exhibit "considerable remapping" when they exceed 0.7
rate_remapping_df['has_considerable_remapping'] = rate_remapping_df['max_axis_distance_from_center'] > considerable_remapping_threshold

high_remapping_cells_only = rate_remapping_df[rate_remapping_df['has_considerable_remapping']] 
high_remapping_cells_only

,neuron_type,laps,replays,skew,max_axis_distance_from_center,distance_from_center,has_considerable_remapping
aclu,,,,,,,
4,NeuronType.PYRAMIDAL,-0.970214,-1.000000,0.970214,1.000000,1.393311,True
89,NeuronType.PYRAMIDAL,-0.191437,-1.000000,0.191437,1.000000,1.018159,True
25,NeuronType.PYRAMIDAL,-0.460861,-0.846932,0.544154,0.846932,0.964203,True
79,NeuronType.CONTAMINATED,-0.158822,-0.748029,0.212321,0.748029,0.764704,True
56,NeuronType.PYRAMIDAL,-0.390185,-0.738485,0.528359,0.738485,0.835227,True
3,NeuronType.PYRAMIDAL,-0.852888,-0.671109,1.270863,0.852888,1.085268,True
13,NeuronType.PYRAMIDAL,-0.883209,-0.612142,1.442816,0.883209,1.074605,True
58,NeuronType.PYRAMIDAL,-0.867777,-0.539014,1.609933,0.867777,1.021554,True
8,NeuronType.PYRAMIDAL,0.946429,-0.200761,-4.714205,0.946429,0.967488,True


In [170]:
rate_remapping_df.to_clipboard()


In [ ]:
from neuropy.core.neurons import NeuronType

# Get color corresponding to each neuron type:
a_neuron_type: NeuronType = rr_neuron_type[0]
a_neuron_type

a_neuron_type.renderColor

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import LongShortTrackComparingDisplayFunctions, plot_rr_aclu

## Single plot
n_debug_limit = 20
fig, ax, sort_indicies = plot_rr_aclu([str(aclu) for aclu in rr_aclus[:n_debug_limit]], rr_laps=rr_laps[:n_debug_limit], rr_replays=rr_replays[:n_debug_limit], rr_neuron_types=rr_neuron_type[:n_debug_limit])

In [ ]:
# Plot using Paginator:
import matplotlib.pyplot as plt
%matplotlib qt

from pyphocorehelpers.indexing_helpers import Paginator
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import RateRemappingPaginatedFigureController
## Paginated multi-plot
# Provide a tuple or list containing equally sized sequences of items:
a_paginator = Paginator.init_from_data((rr_aclus, rr_laps, rr_replays, rr_neuron_type), max_num_columns=1, max_subplots_per_page=20, data_indicies=None, last_figure_subplots_same_layout=False)

## Build GUI components:
active_identifying_session_ctx = curr_active_pipeline.sess.get_context()
_out_rr_pagination_controller = RateRemappingPaginatedFigureController.init_from_paginator(a_paginator, a_name='TestRateRemappingPaginatedFigureController', active_context=active_identifying_session_ctx)
# _out_rr_pagination_controller

In [ ]:
## Not needed, integrated into `RateRemappingPaginatedFigureController` now

from neuropy.utils.misc import split_list_of_dicts
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_figure_basename_from_display_context, session_context_to_relative_path, create_daily_programmatic_display_function_testing_folder_if_needed


## paginated outputs for shared cells
included_unit_indicies_pages = [[curr_included_unit_index for (a_linear_index, curr_row, curr_col, curr_included_unit_index) in v] for page_idx, v in enumerate(a_paginator.included_combined_indicies_pages)] # a list of length `num_pages` containing up to 10 items

active_identifying_session_ctx = curr_active_pipeline.sess.get_context()
paginated_shared_cells_kwarg_list = [dict(active_identifying_ctx=active_identifying_session_ctx.adding_context(collision_prefix='_RateRemapping_plot_test', display_fn_name='plot_rr_aclu', plot_result_set='shared', page=f'{page_idx+1}of{a_paginator.num_pages}', aclus=f"{curr_included_unit_indicies}"),
	fignum=f'shared_{page_idx}', fig_idx=page_idx, n_max_page_rows=a_paginator.max_num_items_per_page) for page_idx, curr_included_unit_indicies in enumerate(included_unit_indicies_pages)]

# Can build a list of keyword arguments that will be provided to the function of interest ahead of time
# paginated_shared_cells_kwarg_list = [dict(included_unit_neuron_IDs=curr_included_unit_indicies,
# 	active_identifying_ctx=active_identifying_session_ctx.adding_context(collision_prefix='_RateRemapping_plot_test', display_fn_name='RateRemapping_plot_test', plot_result_set='shared', page=f'{page_idx+1}of{a_paginator.num_pages}', aclus=f"{curr_included_unit_indicies}"),
# 	fignum=f'shared_{page_idx}', fig_idx=page_idx, n_max_page_rows=a_paginator.max_num_items_per_page) for page_idx, curr_included_unit_indicies in enumerate(included_unit_indicies_pages)]

paginated_dict_of_lists = split_list_of_dicts(paginated_shared_cells_kwarg_list)
active_identifying_ctxs, fignums, fig_idxs = paginated_dict_of_lists['active_identifying_ctx'], paginated_dict_of_lists['fignum'], paginated_dict_of_lists['fig_idx']

# figures_parent_out_path = create_daily_programmatic_display_function_testing_folder_if_needed()
# active_session_figures_out_path = session_context_to_relative_path(figures_parent_out_path, active_identifying_session_ctx)
# print(f'curr_session_parent_out_path: {active_session_figures_out_path}')
# active_session_figures_out_path.mkdir(parents=True, exist_ok=True) # make folder if needed

# # # Add in the desired display variable:
# active_identifying_ctx = active_identifying_display_ctx.adding_context('filter_epochs', **active_display_fn_kwargs) # , filter_epochs='ripple' ## TODO: this is only right for a single function!

active_identifying_ctx = active_identifying_ctxs[0]
final_context = active_identifying_ctx # Display/Variable context mode

active_identifying_ctx_string = final_context.get_description(separator='|') # Get final discription string
print(f'active_identifying_ctx_string: "{active_identifying_ctx_string}"')

active_figure_save_basename = build_figure_basename_from_display_context(final_context)
# _out_rr_pagination_controller.curr_page_index


_out_rr_pagination_controller.update_titles(active_identifying_ctx_string)


In [ ]:

_out_rr_pagination_controller.ui.mw.ui.paginator_controller_widget.current_page_idx
# curr_page_index

In [ ]:

active_identifying_ctx = active_identifying_ctxs[0]
final_context = active_identifying_ctx # Display/Variable context mode

active_identifying_ctx_string = final_context.get_description(separator='|') # Get final discription string
print(f'active_identifying_ctx_string: "{active_identifying_ctx_string}"')

build_figure_basename_from_display_context(final_context)


In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE) # AttributeError: 'PfND_TimeDependent' object has no attribute '_included_thresh_neurons_indx'
# TypeError: cannot pickle 'MplMultiTab' object

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE)

In [91]:
_plot_obj = curr_active_pipeline.plot
# _plot_obj._display_spike_rasters_window()
_plot_obj

In [87]:
curr_active_pipeline.reload_default_display_functions()
curr_active_pipeline.plot._display_jonathan_interactive_replay_firing_rate_comparison


In [95]:
curr_active_pipeline.plot._display_batch_pho_jonathan_replay_firing_rate_comparison()


TypeError: _display_batch_pho_jonathan_replay_firing_rate_comparison() missing 2 required positional arguments: 'computation_results' and 'active_configs'

In [93]:
curr_active_pipeline.plot._display_short_long_firing_rate_index_comparison()


TypeError: _display_short_long_firing_rate_index_comparison() missing 2 required positional arguments: 'computation_results' and 'active_configs'

In [96]:
active_identifying_session_ctx, active_session_figures_out_path, active_out_figures_list = batch_programmatic_figures(curr_active_pipeline)

[autoreload of pyphoplacecellanalysis.General.Batch.NonInteractiveWrapper failed: Traceback (most recent call last):
  File "c:\Users\pho\AppData\Local\pypoetry\Cache\virtualenvs\spike3d-WtdfU5rp-py3.9\lib\site-packages\IPython\extensions\autoreload.py", line 271, in check
    superreload(m, reload, self.old_objects, self.shell)
  File "c:\Users\pho\AppData\Local\pypoetry\Cache\virtualenvs\spike3d-WtdfU5rp-py3.9\lib\site-packages\IPython\extensions\autoreload.py", line 496, in superreload
    update_generic(old_obj, new_obj)
  File "c:\Users\pho\AppData\Local\pypoetry\Cache\virtualenvs\spike3d-WtdfU5rp-py3.9\lib\site-packages\IPython\extensions\autoreload.py", line 393, in update_generic
    update(a, b)
  File "c:\Users\pho\AppData\Local\pypoetry\Cache\virtualenvs\spike3d-WtdfU5rp-py3.9\lib\site-packages\IPython\extensions\autoreload.py", line 361, in update_class
    update_instances(old, new)
  File "c:\Users\pho\AppData\Local\pypoetry\Cache\virtualenvs\spike3d-WtdfU5rp-py3.9\lib\si

shared_aclus: [3, 5, 6, 7, 9, 10, 11, 12, 14, 15, 17, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 106, 107, 108]
long_only_aclus: [2, 8, 16, 19, 105, 109]
short_only_aclus: [4, 13, 23, 58]
curr_session_parent_out_path: C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2023-05-12\kdiba\gor01\one\2006-6-08_14-26-15
shared_aclus: [3, 5, 6, 7, 9, 10, 11, 12, 14, 15, 17, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92

C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\General\Pipeline\Stages\DisplayFunctions\MultiContextComparingDisplayFunctions\LongShortTrackComparingDisplayFunctions.py:679: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cell_df.loc[:, 'x'] = spk_x
C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\General\Mixins\CrossComputationComparisonHelpers.py:208: RuntimeWarning: invalid value encountered in divide
  _long_to_short_balances = (rdf.num_short_only_neuron_participating.values - rdf.num_long_only_neuron_participating.values) / (rdf.num_short_only_neuron_participating.values + rdf.num_long_only_neuron_participating.values) ## Working but (0, 1, 1) would clip to 0.5 despite (1,

C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\General\Pipeline\Stages\DisplayFunctions\MultiContextComparingDisplayFunctions\LongShortTrackComparingDisplayFunctions.py:913: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  fig.show()


	 saved C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2023-05-12\kdiba\gor01\one\2006-6-08_14-26-15\kdiba_gor01_one_2006-6-08_14-26-15_batch_plot_test_long_only_[2, 8, 16, 19, 105, 109].png
include_whitelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze


...

C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\General\Pipeline\Stages\DisplayFunctions\MultiContextComparingDisplayFunctions\LongShortTrackComparingDisplayFunctions.py:913: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  fig.show()


	 saved C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2023-05-12\kdiba\gor01\one\2006-6-08_14-26-15\kdiba_gor01_one_2006-6-08_14-26-15_batch_plot_test_shared_9of10_[90, 91, 92, 93, 94, 95, 96, 97, 98, 99].png
include_whitelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze


C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\General\Pipeline\Stages\DisplayFunctions\MultiContextComparingDisplayFunctions\LongShortTrackComparingDisplayFunctions.py:679: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cell_df.loc[:, 'x'] = spk_x
C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\General\Mixins\CrossComputationComparisonHelpers.py:208: RuntimeWarning: invalid value encountered in divide
  _long_to_short_balances = (rdf.num_short_only_neuron_participating.values - rdf.num_long_only_neuron_participating.values) / (rdf.num_short_only_neuron_participating.values + rdf.num_long_only_neuron_participating.values) ## Working but (0, 1, 1) would clip to 0.5 despite (1,

C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\General\Pipeline\Stages\DisplayFunctions\MultiContextComparingDisplayFunctions\LongShortTrackComparingDisplayFunctions.py:913: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  fig.show()


	 saved C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2023-05-12\kdiba\gor01\one\2006-6-08_14-26-15\kdiba_gor01_one_2006-6-08_14-26-15_batch_plot_test_shared_10of10_[100, 101, 102, 103, 104, 106, 107, 108].png
Saving plot_long_short_firing_rate_indicies results results : C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2023-05-12\kdiba\gor01\one\2006-6-08_14-26-15\kdiba_gor01_one_2006-6-08_14-26-15_long_short_firing_rate_indicies.png... done.


In [ ]:

curr_active_pipeline.plot._display_context_nested_docks

_display_short_long_firing_rate_index_comparison

In [98]:
from pyphocorehelpers.programming_helpers import IPythonHelpers

long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
x_frs_index, y_frs_index = long_short_fr_indicies_analysis_results['x_frs_index'], long_short_fr_indicies_analysis_results['y_frs_index'] # use the all_results_dict as the computed data value
active_context = long_short_fr_indicies_analysis_results['active_context']

IPythonHelpers.cell_vars(lambda: globals())

{'long_short_fr_indicies_analysis_results': DynamicParameters({'long_laps': 35 epochs
 array([[   5.73892314,   13.90835   ],
        [  30.53504997,   39.76855564],
        [ 135.83449301,  144.1481502 ],
        [ 161.5538168 ,  167.31814537],
        [ 234.47012635,  239.73553166],
        [ 255.25081331,  262.68154743],
        [ 294.55684887,  299.86354183],
        [ 313.88371257,  319.41764126],
        [ 499.2982634 ,  504.74141453],
        [ 511.58078895,  518.38704723],
        [ 558.24625754,  565.53211621],
        [ 584.28583717,  591.15980229],
        [ 599.27239717,  604.70150927],
        [ 616.77704357,  625.65610206],
        [ 692.41993604,  697.52098943],
        [ 712.59417979,  721.66861739],
        [ 734.30912291,  740.66232286],
        [ 750.56020963,  755.13588985],
        [ 763.15076447,  768.97127989],
        [ 783.13030496,  787.93021034],
        [ 804.52732696,  812.14480786],
        [ 825.79335895,  832.09866645],
        [ 848.69077571,  853.68673

In [119]:
long_short_fr_indicies_df = pd.DataFrame({'aclu': list(x_frs_index.keys()), 'replay_fr_idx': list(x_frs_index.values()), 'laps_fr_idx': list(y_frs_index.values())}) # , columns=['aclu','replay_frs']
long_short_fr_indicies_df.set_index('aclu', inplace=True)
# long_short_fr_indicies_df
long_short_fr_indicies_df.to_clipboard()

In [99]:
long_short_fr_indicies_analysis_results

DynamicParameters({'long_laps': 35 epochs
array([[   5.73892314,   13.90835   ],
       [  30.53504997,   39.76855564],
       [ 135.83449301,  144.1481502 ],
       [ 161.5538168 ,  167.31814537],
       [ 234.47012635,  239.73553166],
       [ 255.25081331,  262.68154743],
       [ 294.55684887,  299.86354183],
       [ 313.88371257,  319.41764126],
       [ 499.2982634 ,  504.74141453],
       [ 511.58078895,  518.38704723],
       [ 558.24625754,  565.53211621],
       [ 584.28583717,  591.15980229],
       [ 599.27239717,  604.70150927],
       [ 616.77704357,  625.65610206],
       [ 692.41993604,  697.52098943],
       [ 712.59417979,  721.66861739],
       [ 734.30912291,  740.66232286],
       [ 750.56020963,  755.13588985],
       [ 763.15076447,  768.97127989],
       [ 783.13030496,  787.93021034],
       [ 804.52732696,  812.14480786],
       [ 825.79335895,  832.09866645],
       [ 848.69077571,  853.68673603],
       [ 923.92813367,  933.59405384],
       [ 941.8437213 ,

# 2023-03-16 - Explore passing in long/short decoders specifically to `perform_full_session_leave_one_out_decoding_analysis`

In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import build_neurons_color_map # for plot_short_v_long_pf1D_comparison

# Get existing long/short decoders from the cell under "# 2023-02-24 Decoders"
long_decoder, short_decoder = deepcopy(long_one_step_decoder_1D), deepcopy(short_one_step_decoder_1D)
assert np.all(long_decoder.xbin == short_decoder.xbin)

## 2023-05-01 - Test my interneuron hypothesis by looking at interneurons across the long/short divide

 1. Interneuron identity should be best determinant of active environment.

 2. Do people know anything about interneurons that activate in a given environment? For example, to interneurons activate alongside pyramidal cells when replaying a given environment?

 3. 


In [ ]:
# global_results.pf1D_dt.reset()
# global_results.pf2D_dt.reset()
# global_results.all_attributes

list(curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.keys())

curr_active_pipeline.global_computation_results.computed_data.long_short

In [ ]:


# 1. Interneuron identity should be best determinant of active environment.

# 2. Do people know anything about interneurons that activate in a given environment? For example, to interneurons activate alongside pyramidal cells when replaying a given environment?

# 3. 
# global_results.pf1D_dt.reset()
# global_results.pf2D_dt.reset()
# global_results.all_attributes

list(curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.keys())

curr_active_pipeline.global_computation_results.computed_data.long_short

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.MultiContextComputationFunctions import JonathanFiringRateAnalysisResult

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())

jonathan_firing_rate_analysis_result.neuron_replay_stats_df.to_clipboard()

In [ ]:
jonathan_firing_rate_analysis_result.rdf.rdf

In [ ]:
jonathan_firing_rate_analysis_result.time_binned_instantaneous_unit_specific_spike_rate.time_bins #.shape

In [ ]:
instantaneous_fr_df = deepcopy(jonathan_firing_rate_analysis_result.time_binned_instantaneous_unit_specific_spike_rate.instantaneous_unit_specific_spike_rate_values)
instantaneous_fr_df['t'] = jonathan_firing_rate_analysis_result.time_binned_instantaneous_unit_specific_spike_rate.time_bins
instantaneous_fr_df.plot


In [ ]:
instantaneous_fr_df.to_clipboard()

In [ ]:
from neuropy.core.neurons import NeuronType

In [ ]:
jonathan_firing_rate_analysis_result.neuron_replay_stats_df

In [ ]:
## Require placefield presence on either the long or the short
jonathan_firing_rate_analysis_result.neuron_replay_stats_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df[np.logical_or(jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_long_pf'], jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_short_pf'])]

In [ ]:
jonathan_firing_rate_analysis_result.neuron_replay_stats_df[np.logical_or(jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_long_pf'], jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_short_pf'])].index.to_numpy()

In [ ]:
jonathan_firing_rate_analysis_result.neuron_replay_stats_df.to_clipboard()

In [ ]:
from pyphocorehelpers.indexing_helpers import partition

# Split the replay stats based on the neuron types:
unique_included_neuron_types, (pyr_neuron_replay_stats_df, contaminated_neuron_replay_stats_df, interneuron_neuron_replay_stats_df) = partition(jonathan_firing_rate_analysis_result.neuron_replay_stats_df, 'neuron_type')
pyr_neuron_replay_stats_df

In [ ]:
from neuropy.core.neurons import NeuronType

%matplotlib qt
# Look at replays during ripples vs. those not during ripples. Also potentially PBEs vs. not PBEs.

# NeuronType.from_any_string_series(['pyr','intr'])
'pyr','cont','intr'

active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'
curr_active_pipeline.display('_display_jonathan_interactive_replay_firing_rate_comparison', active_identifying_session_ctx, included_neuron_types=NeuronType.from_any_string_series(['pyr'])) # only the pyramidal neurons

In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [ ]:
curr_active_pipeline.display('_display_jonathan_interactive_replay_firing_rate_comparison', active_identifying_session_ctx, included_neuron_types=NeuronType.from_any_string_series(['intr']), require_placefield=False) # only the pyramidal neurons

In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import build_replays_custom_scatter_markers, CustomScatterMarkerMode

_curr_included_aclus = jonathan_firing_rate_analysis_result.neuron_replay_stats_df[np.logical_or(jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_long_pf'], jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_short_pf'])].index.to_numpy()
_curr_output = curr_active_pipeline.display('_display_batch_pho_jonathan_replay_firing_rate_comparison', active_identifying_session_ctx, included_unit_neuron_IDs=_curr_included_aclus, marker_split_mode=CustomScatterMarkerMode.NoSplit)

In [ ]:
_curr_fig = _curr_output['fig']
_curr_axes = _curr_output['axs']

for ax_dict in _curr_axes:
    for k, ax in ax_dict.items():
	    ax.set_rasterized(True)
    
# _curr_fig.set_rasterized(True)

## 2023-04-13 - Shuffled Surprise
""" 
Relevant Functions:
`perform_full_session_leave_one_out_decoding_analysis`:
	`perform_leave_one_aclu_out_decoding_analysis`:	from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import perform_leave_one_aclu_out_decoding_analysis
	`_analyze_leave_one_out_decoding_results`: from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.DefaultComputationFunctions import _analyze_leave_one_out_decoding_results
"""


In [ ]:
# 2023-04-18 - Refactored into decoder_result
result, result_df, result_df_grouped = long_results_obj.result, long_results_obj.result_df, long_results_obj.result_df_grouped

In [ ]:
curr_active_pipeline.global_computation_results

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import DiagnosticDistanceMetricFigure

## Render the internactive slider that allows selecting the timebin index to debug
n_timebins = np.sum(long_results_obj.all_epochs_num_epoch_time_bins)
timebinned_neuron_info = long_results_obj.timebinned_neuron_info
result = long_results_obj.new_result
active_fig_obj, update_function = DiagnosticDistanceMetricFigure.build_interactive_diagnostic_distance_metric_figure(long_results_obj, timebinned_neuron_info, result)
active_fig_obj.integer_slider(n_timebins=n_timebins, update_func=update_function)

In [ ]:

# 1. Get set of cells active in a given time bin, for each compute the surprise of its placefield with the leave-one-out decoded posterior.

# 2. From the remainder of cells (those not active), randomly choose one to grab the placefield of and compute the surprise with that and the same posterior.

# Expectation: The cells that are included in the time bin are expected to have a lower surprise (be less correlated with) the posterior.

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.Extensions.pyqtgraph_helpers import build_pyqtgraph_epoch_indicator_regions # Add session indicators to pyqtgraph plot
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.MultiContextComparingDisplayFunctions import _helper_add_long_short_session_indicator_regions

from pyphocorehelpers.print_helpers import generate_html_string
# Long Short
# TODO 2023-04-18 - Can Refactor in terms of `plot_long_short_any_values`?

import pyphoplacecellanalysis.External.pyqtgraph as pg
# 'time_bin_indices': valid_time_bin_indicies, 'posterior_to_pf_mean_surprise': one_left_out_posterior_to_pf_surprises_mean, 'posterior_to_scrambled_pf_mean_surprise': one_left_out_posterior_to_scrambled_pf_surprises_mean}
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.MultiContextComparingDisplayFunctions import plot_long_short_expected_vs_observed_firing_rates


In [ ]:


win, plots_tuple, legend = plot_long_short_expected_vs_observed_firing_rates(long_results_obj, short_results_obj)
long_epoch_indicator_region_items, short_epoch_indicator_region_items = _helper_add_long_short_session_indicator_regions(win, long_epoch=curr_active_pipeline.filtered_epochs[long_epoch_name], short_epoch=curr_active_pipeline.filtered_epochs[short_epoch_name])

In [ ]:


def plot_long_short_surprise_difference_plot(long_results_obj, short_results_obj):
	""" captures `curr_active_pipeline` """
	# Private Subfunctions _______________________________________________________________________________________________ #
	def _subfn_add_difference_plot_series(win, plots, result_df_grouped, series_suffix, **kwargs):
		""" captures nothing
		modifies `plots` """
		x=result_df_grouped.time_bin_centers.to_numpy()
		y=result_df_grouped['surprise_diff'].to_numpy()
		series_id_str = f'difference_{series_suffix}'
		plots[series_id_str] = win.plot(x=x, y=y, name=series_id_str, alpha=0.5, **kwargs) #  symbolBrush=pg.intColor(i,6,maxValue=128) , symbol=curr_symbol, symbolBrush=cell_color_symbol_brush[unit_IDX]

	# BEGIN FUNCTION BODY ________________________________________________________________________________________________ #

	# make a separate symbol_brush color for each cell:
	# cell_color_symbol_brush = [pg.intColor(i,hues=9, values=3, alpha=180) for i, aclu in enumerate(long_results_obj.original_1D_decoder.neuron_IDs)] # maxValue=128
	# All properties in common:
	win = pg.plot() # PlotWidget
	win.setWindowTitle('Long Sanity Check - Leave-one-out Custom Surprise Plot')
	# legend_size = (80,60) # fixed size legend
	legend_size = None # auto-sizing legend to contents
	legend = pg.LegendItem(legend_size, offset=(-1,0)) # do this instead of # .addLegend
	legend.setParentItem(win.graphicsItem())

	plots = {}
	label_prefix_list = ['normal', 'scrambled']
	long_short_symbol_list = ['t', 'o'] # note: 's' is a square. 'o', 't1': triangle pointing upwards0

	# Use mean time_bin and surprise for each epoch
	# plots['normal'] = win.plot(x=valid_time_bin_indicies, y=one_left_out_posterior_to_pf_surprises_mean, pen=None, symbol='t', symbolBrush=pg.intColor(1,6,maxValue=128), name=f'normal', alpha=0.5) #  symbolBrush=pg.intColor(i,6,maxValue=128) , symbol=curr_symbol, symbolBrush=cell_color_symbol_brush[unit_IDX]
	# plots['scrambled'] = win.plot(x=valid_time_bin_indicies, y=one_left_out_posterior_to_scrambled_pf_surprises_mean, pen=None, symbol='t', symbolBrush=pg.intColor(2,6,maxValue=128), name=f'scrambled', alpha=0.5) #  symbolBrush=pg.intColor(i,6,maxValue=128) , symbol=curr_symbol, symbolBrush=cell_color_symbol_brush[unit_IDX]

	# curr_surprise_difference = one_left_out_posterior_to_scrambled_pf_surprises_mean - one_left_out_posterior_to_pf_surprises_mean

	# x=valid_time_bin_indicies
	# y=curr_surprise_difference
	# x=result_df_grouped.time_bin_indices.to_numpy()

	
	_subfn_add_difference_plot_series(win, plots, long_results_obj.result_df_grouped, series_suffix='_long', **dict(pen=None, symbol='t', symbolBrush=pg.intColor(2,6,maxValue=128), clickable=True, hoverable=True, hoverSize=7))

	_subfn_add_difference_plot_series(win, plots, short_results_obj.result_df_grouped, series_suffix='_short', **dict(pen=None, symbol='o', symbolBrush=pg.intColor(3,6,maxValue=128), clickable=True))

	# dict(pen=None, symbol='t', symbolBrush=pg.intColor(2,6,maxValue=128))


	# x=result_df_grouped.time_bin_centers.to_numpy()
	# y=result_df_grouped['surprise_diff'].to_numpy()
	# plots['difference'] = win.plot(x=x, y=y, pen=None, symbol='t', symbolBrush=pg.intColor(2,6,maxValue=128), name=f'difference', alpha=0.5) #  symbolBrush=pg.intColor(i,6,maxValue=128) , symbol=curr_symbol, symbolBrush=cell_color_symbol_brush[unit_IDX]

	# long_results_obj.result, long_results_obj.result_df, long_results_obj.result_df_grouped

	# short_results_obj.result, short_results_obj.result_df, short_results_obj.result_df_grouped


	for k, v in plots.items():
		legend.addItem(v, f'{k}')

	win.graphicsItem().setLabel(axis='left', text='Normal v. Random - Surprise (Custom)')
	win.graphicsItem().setLabel(axis='bottom', text='time')

	win.showGrid(True, True)  # Show grid for reference

	# Emphasize the y=0 crossing by drawing a horizontal line at y=0
	vline = pg.InfiniteLine(pos=0, angle=0, movable=False, pen=pg.mkPen(color='w', width=2, style=pg.QtCore.Qt.DashLine))
	win.addItem(vline)

	# Add session indicators to pyqtgraph plot
	long_epoch = curr_active_pipeline.filtered_epochs[long_epoch_name]
	short_epoch = curr_active_pipeline.filtered_epochs[short_epoch_name]
	long_epoch_indicator_region_items, short_epoch_indicator_region_items = _helper_add_long_short_session_indicator_regions(win, long_epoch, short_epoch)

	# epoch_linear_region, epoch_region_label = build_pyqtgraph_epoch_indicator_regions(win, t_start=curr_active_pipeline.filtered_epochs[long_epoch_name].t_start, t_stop=curr_active_pipeline.filtered_epochs[long_epoch_name].t_stop, epoch_label='long', **dict(pen=pg.mkPen('#0b0049'), brush=pg.mkBrush('#0099ff42'), hoverBrush=pg.mkBrush('#fff400'), hoverPen=pg.mkPen('#00ff00')))
	# epoch_linear_region, epoch_region_label = build_pyqtgraph_epoch_indicator_regions(win, t_start=curr_active_pipeline.filtered_epochs[short_epoch_name].t_start, t_stop=curr_active_pipeline.filtered_epochs[short_epoch_name].t_stop, epoch_label='short', **dict(pen=pg.mkPen('#490000'), brush=pg.mkBrush('#f5161659'), hoverBrush=pg.mkBrush('#fff400'), hoverPen=pg.mkPen('#00ff00')))

	i_str = generate_html_string('i', color='white', bold=True)
	j_str = generate_html_string('j', color='red', bold=True)
	title_str = generate_html_string(f'JSD(p_x_given_n, pf[{i_str}]) - JSD(p_x_given_n, pf[{j_str}]) where {j_str} non-firing')
	win.setTitle(title_str)

	win.setWindowTitle('Long Sanity Check - Leave-one-out Custom Surprise Plot - JSD')

	return win, plots

win, self.plots = plot_long_short_surprise_difference_plot(long_results_obj, short_results_obj)

In [ ]:
a_plot = self.plots['difference__long']  # PlotDataItem 
a_curve = a_plot.curve # PlotCurveItem 

In [ ]:
a_plot.sigPointsClicked
# a_plot.sigPointsHovered

In [ ]:
curve.curve.setClickable(True)

In [ ]:
# Will make all plots clickable
clickedPen = pg.mkPen('#DDD', width=2)
lastClicked = []
def _test_scatter_plot_clicked(plot, points):
	global lastClicked
	for p in lastClicked:
		p.resetPen()
	print("clicked points", points)
	for p in points:
		p.setPen(clickedPen)
	lastClicked = points

main_scatter_clicked_connection = a_plot.sigClicked.connect(_test_scatter_plot_clicked)

In [ ]:
lastClicked, clickedPen, (main_scatter_hovered_connection, main_scatter_clicked_connection) = _helper_make_scatterplot_clickable(a_plot)

In [ ]:
# Need to map from time_bin_indicies to times
a_plot = self.plots['difference'] # PlotDataItem 
# a_plot.setLa
# win.graphicsItem().setLabel(axis='left', text='Normal v. Random - Surprise (Custom)')

# # Set the plot title with a LaTeX formula
# title = pg.LabelItem(justify='center')
# title.setText(r'<font size="4">JSD Surprise Diff: $\fn{JSD}{\matr{pf}_{i}, \Pr{\vec{x}_{t}|\vec{n}_{t}}} - \fn{JSD}{\matr{pf}_{j}, \Pr{\vec{x}_{t}|\vec{n}_{t}}}$</font>')
# win.addItem(title)

# win.graphicsItem().setLabel(axis='top', text=r'$\fn{JSD}{\matr{pf}_{i}, \Pr{\vec{x}_{t}|\vec{n}_{t}}} - \fn{JSD}{\matr{pf}_{j}, \Pr{\vec{x}_{t}|\vec{n}_{t}}}$')

In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot # works pretty well seemingly

export_pyqtgraph_plot(win)
# pg.setConfigOption('leftTitle', 'MathText')
# win.setTitle(r'JSD(p_x_given_n, pf[<font size="4"><b><span style="color:red;">i</span></b></font>]) - JSD(p_x_given_n, pf[<font size="4"><b>j</b></font>]) where <font size="4"><b>j</b></font> non-firing')

# win.setTitle(generate_html_string(f'JSD(p_x_given_n, pf[{i_str}]) - JSD(p_x_given_n, pf[{j_str}]) where {j_str} non-firing', font_size=8))

# r'$\fn{JSD}{\matr{pf}_{i}, \Pr{\vec{x}_{t}|\vec{n}_{t}}} - \fn{JSD}{\matr{pf}_{j}, \Pr{\vec{x}_{t}|\vec{n}_{t}}}$'
# title_str

In [ ]:
from pyqtgraph.GraphicsScene import exportDialog

# active_item = active_fig_obj.win
active_item = active_fig_obj.plot_dict['curr_cell_pf_curve']['plot_item']
exportDialog = exportDialog.ExportDialog(active_item.scene())
exportDialog.show(active_item)

In [ ]:
active_fig_obj.plot_dict['curr_cell_pf_curve']['plot_item']

In [ ]:
curr_active_pipeline.filtered_epochs[short_epoch_name].t_stop

In [ ]:
curr_active_pipeline.filtered_epochs[long_epoch_name].start_end_times

## Pre 2023-04-13

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib qt

from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import plot_kourosh_activity_style_figure

from neuropy.core.neurons import NeuronType
# # Include only pyramidal aclus:
# print(f'all shared_aclus: {len(shared_aclus)}\nshared_aclus: {shared_aclus}')
# shared_aclu_neuron_type = long_session.neurons.neuron_type[np.isin(long_session.neurons.neuron_ids, shared_aclus)]
# assert len(shared_aclu_neuron_type) == len(shared_aclus)
# # Find only the aclus that are pyramidal:
# is_shared_aclu_pyramidal = (shared_aclu_neuron_type == NeuronType.PYRAMIDAL)
# pyramidal_only_shared_aclus = shared_aclus[is_shared_aclu_pyramidal]
# print(f'num pyramidal_only_shared_aclus: {len(pyramidal_only_shared_aclus)}\npyramidal_only_shared_aclus: {pyramidal_only_shared_aclus}')


## Drop Pyramidal but don't use only shared aclus:
all_aclus = deepcopy(long_session.neurons.neuron_ids)
neuron_type = long_session.neurons.neuron_type
assert len(neuron_type) == len(all_aclus)
# Find only the aclus that are pyramidal:
is_aclu_pyramidal = (neuron_type == NeuronType.PYRAMIDAL)
pyramidal_only_all_aclus = all_aclus[is_aclu_pyramidal]
print(f'num pyramidal_only_all_aclus: {len(pyramidal_only_all_aclus)}\npyramidal_only_all_aclus: {pyramidal_only_all_aclus}')

is_aclu_interneuron = (neuron_type == NeuronType.INTERNEURONS)
interneuron_only_all_aclus = all_aclus[is_aclu_interneuron]
print(f'num interneuron_only_all_aclus: {len(interneuron_only_all_aclus)}\ninterneurons: {interneuron_only_all_aclus}')

# app, win, plots, plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, shared_aclus, epoch_idx=5, callout_epoch_IDXs=[0,1,2,3], skip_rendering_callouts=True)
# app, win, plots, plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, pyramidal_only_shared_aclus, epoch_idx=2, callout_epoch_IDXs=[0,4], skip_rendering_callouts=False)
app, win, self.plots, self.plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, pyramidal_only_all_aclus, epoch_idx=6, callout_epoch_IDXs=[2,4,6], skip_rendering_callouts=False)

In [ ]:

# Use the Jupyter Index Thing
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import DiagnosticDistanceMetricFigure

## Render the internactive slider that allows selecting the timebin index to debug
n_timebins = np.sum(long_results_obj.all_epochs_num_epoch_time_bins)
timebinned_neuron_info = long_results_obj.timebinned_neuron_info
result = long_results_obj.new_result
active_fig_obj, update_function = DiagnosticDistanceMetricFigure.build_interactive_diagnostic_distance_metric_figure(long_results_obj, timebinned_neuron_info, result)
active_fig_obj.integer_slider(n_timebins=n_timebins, update_func=update_function)

In [ ]:
active_fig_obj.export()

In [ ]:
app, win, self.plots, self.plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, pyramidal_only_all_aclus, epoch_idx=11, callout_epoch_IDXs=[0,1,2, 3, 4, 5], skip_rendering_callouts=False)

# 2023-04-13 Show Surprise 

In [ ]:
import pyphoplacecellanalysis.External.pyqtgraph as pg
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.MultiContextComparingDisplayFunctions import plot_long_short, plot_long_short_any_values, plot_long_short_expected_vs_observed_firing_rates, _helper_add_long_short_session_indicator_regions
# plot_long_short(long_results_obj, short_results_obj)

In [ ]:
plot_long_short_any_values(long_results_obj=long_results_obj, short_results_obj=short_results_obj)

In [ ]:
plot_long_short_expected_vs_observed_firing_rates(long_results_obj=long_results_obj, short_results_obj=short_results_obj, limit_aclus=[89]) # 4, 89, 28, 97

In [ ]:
x_fn = lambda a_results_obj: a_results_obj.all_epochs_decoded_epoch_time_bins_mean[:,0]
# y_fn = lambda a_results_obj: a_results_obj.all_epochs_all_cells_one_left_out_posterior_to_scrambled_pf_surprises_mean
y_fn = lambda a_results_obj: a_results_obj.all_epochs_all_cells_one_left_out_posterior_to_pf_surprises_mean
# y_fn = lambda a_results_obj: a_results_obj.all_epochs_computed_one_left_out_posterior_to_pf_surprises

# (time_bins, neurons), (epochs, neurons), (epochs)
# all_epochs_computed_one_left_out_posterior_to_pf_surprises, all_epochs_computed_cell_one_left_out_posterior_to_pf_surprises_mean, all_epochs_all_cells_one_left_out_posterior_to_pf_surprises_mean
win, plots_tuple, legend = plot_long_short_any_values(long_results_obj, short_results_obj, x=x_fn, y=y_fn, limit_aclus=[20])

# 2023-04-13 - Find Good looking epochs:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib qt

from pyphocorehelpers.plotting.figure_management import PhoActiveFigureManager2D

In [ ]:
from pyphoplacecellanalysis.GUI.Qt.Mixins.PaginationMixins import PaginatedFigureController
from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController

active_identifying_session_ctx = curr_active_pipeline.sess.get_context()
_out_pagination_controller = DecodedEpochSlicesPaginatedFigureController.init_from_decoder_data(long_results_obj.active_filter_epochs, long_results_obj.all_included_filter_epochs_decoder_result, 
	xbin=long_results_obj.original_1D_decoder.xbin, global_pos_df=global_session.position.df, a_name='TestDecodedEpochSlicesPaginationController', active_context=active_identifying_session_ctx,  max_subplots_per_page=10)
# _out_pagination_controller

In [ ]:

_out_pagination_controller.selected_indicies


In [ ]:
# .

current_page_idx = _out_pagination_controller.ui.mw.ui.paginator_controller_widget.current_page_idx
curr_page_data_indicies, curr_page_data_items =  _out_pagination_controller.plots_data.paginator.get_page_data(page_idx=current_page_idx)
curr_page_data_indicies

In [ ]:
from pyphocorehelpers.indexing_helpers import safe_find_index_in_list

def _get_current_page_data_indicies():
	""" captures `ui.mw.ui.paginator_controller_widget.current_page_idx`, `plots_data.paginator` """
	current_page_idx = _out_pagination_controller.ui.mw.ui.paginator_controller_widget.current_page_idx
	curr_page_data_indicies, curr_page_data_items =  _out_pagination_controller.plots_data.paginator.get_page_data(page_idx=current_page_idx)
	return current_page_idx, curr_page_data_indicies

In [ ]:
from PendingNotebookCode import PaginatedSelectionManager


sel_man = PaginatedSelectionManager(axes=_out_pagination_controller.plots.axs, fig=_out_pagination_controller.plots.fig)

def on_page_change(updated_page_idx):
    # print(f'on_page_change(updated_page_idx: {updated_page_idx})')
    # Update on page change:
    sel_man.perform_update()

ui.mw.ui.paginator_controller_widget.jump_to_page.connect(on_page_change)

In [ ]:
sel_man.is_selected

In [ ]:
from PendingNotebookCode import SelectionManager

# Create a SelectionManager instance
sm = SelectionManager(self.plots.axs)

In [ ]:
sm.is_selected

In [ ]:
posterior_containers = filter_epochs_decoder_result.marginal_x_list.copy()

[a_marginal_x_list.p_x_given_n for a_marginal_x_list in filter_epochs_decoder_result.marginal_x_list]
[a_marginal_x_list.most_likely_positions_1D for a_marginal_x_list in filter_epochs_decoder_result.marginal_x_list]

posterior_containers
# posterior_containers = np.array(filter_epochs_decoder_result.marginal_x_list.copy())

In [ ]:
included_page_data_indicies, (curr_page_active_filter_epochs, curr_page_epoch_labels, curr_page_time_bin_containers, curr_page_posterior_containers) = epoch_slices_paginator.get_page_data(page_idx=1)
# included_page_data_indicies, (curr_page_active_filter_epochs, curr_page_epoch_labels, curr_page_time_bin_containers) = epoch_slices_paginator.get_page_data(page_idx=1)
included_page_data_indicies
curr_page_posterior_containers[0].p_x_given_n


In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import DecodedFilterEpochsResult
from pyphocorehelpers.indexing_helpers import safe_len

# {k:safe_len(v) for k,v in filter_epochs_decoder_result.__dict__.items() if (safe_len(v) or 0) > 0}

## Only get the items that are num_filter_epochs long:
epoch_splittable_items = {k:v for k,v in filter_epochs_decoder_result.__dict__.items() if (safe_len(v) or 0) == filter_epochs_decoder_result.num_filter_epochs}
# epoch_splittable_items
epoch_splittable_item_names = list(epoch_splittable_items.keys())
# print(epoch_splittable_item_names)
# ('most_likely_positions_list', 'p_x_given_n_list', 'marginal_x_list', 'marginal_y_list', 'most_likely_position_indicies_list', 'spkcount', 'nbins', 'time_bin_containers', 'time_bin_edges')
# epoch_splittable_item_values = [epoch_splittable_items[k] for k in ('most_likely_positions_list', 'p_x_given_n_list', 'marginal_x_list', 'marginal_y_list', 'most_likely_position_indicies_list', 'nbins', 'time_bin_containers', 'time_bin_edges')]
epoch_splittable_item_values = [epoch_splittable_items[k] for k in ('epoch_description_list','time_bin_containers','marginal_x_list')]
# epoch_splittable_item_values

In [ ]:
# [23, 27, 29, ]
[16, 17, 18, 20, 21, 22, 23, 25, 26, 29]


# 2023-03-28 - Playing around with older computations/visualizations from the `_display_short_long_firing_rate_index_comparison` era:


2023-04-20 - Encountered issue with the replays in session '2006-6-08_14-26-15' where they are duplicated exactly twice, like the first half of the rows are legitimate entries and the second half are directly repeated versions of the first with the only difference appearing to be the 'epoch_id' column changes from 1 to 2. 'rel_id' column seems incorrect but different for some reason. It must be how the MATLAB script exports the values.

Also when I'm looking at only the `short_session.replay` there are many non-2 'epoch_id' values, which is strange. 

TODO: It could have something to do with Jonathan's code maybe? Because the 'replay_r' and 'replay_p' columns he added are different. SEEMS FALSE. It's this way even without running Jonathan's code, although the values might have been saved later?
	Also flat_replay_idx jumps from 689 to 1087 at the transition from epoch_id 1 to 2

UPDATE: the 'replay_r' and 'replay_p' columns aren't from Joanthan, they're in the original .replay_info.mat that's imported!


# Plot long|short firing rate index using 'long_short_fr_indicies_analyses':

In [ ]:
# fig_save_parent_path = Path(r'E:\Dropbox (Personal)\Active\Kamran Diba Lab\Results from 2023-01-20 - LongShort Firing Rate Indicies')
## Get the output path (active_session_figures_out_path) for this session (and all of its filtered_contexts as well):
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'
figures_parent_out_path = create_daily_programmatic_display_function_testing_folder_if_needed()
active_session_figures_out_path = session_context_to_relative_path(figures_parent_out_path, active_identifying_session_ctx)
print(f'curr_session_parent_out_path: {active_session_figures_out_path}')
active_session_figures_out_path.mkdir(parents=True, exist_ok=True) # make folder if needed
curr_active_pipeline.display('_display_short_long_firing_rate_index_comparison', curr_active_pipeline.sess.get_context(), fig_save_parent_path=active_session_figures_out_path)
# plt.close() # closes the current figure

In [120]:
curr_active_pipeline.display('_display_short_long_firing_rate_index_comparison', long_short_fr_indicies_analysis_results['active_context'], fig_save_parent_path=None)


Saving plot_long_short_firing_rate_indicies results results : c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\kdiba_gor01_one_2006-6-08_14-26-15_long_short_firing_rate_indicies.png... done.


[autoreload of pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions failed: Traceback (most recent call last):
  File "c:\Users\pho\AppData\Local\pypoetry\Cache\virtualenvs\spike3d-WtdfU5rp-py3.9\lib\site-packages\IPython\extensions\autoreload.py", line 271, in check
    superreload(m, reload, self.old_objects, self.shell)
  File "c:\Users\pho\AppData\Local\pypoetry\Cache\virtualenvs\spike3d-WtdfU5rp-py3.9\lib\site-packages\IPython\extensions\autoreload.py", line 496, in superreload
    update_generic(old_obj, new_obj)
  File "c:\Users\pho\AppData\Local\pypoetry\Cache\virtualenvs\spike3d-WtdfU5rp-py3.9\lib\site-packages\IPython\extensions\autoreload.py", line 393, in update_generic
    update(a, b)
  File "c:\Users\pho\AppData\Local\pypoetry\Cache\virtualenvs\spike3d-WtdfU5rp-py3.9\lib\site-packages\IPython\extensions\autoreload.py", line 361, in update_class
    update_instances(old, new)
  File "

MatplotlibRenderPlots({'name': 'display_short_long_firing_rate_index_comparison', 'figures': <Figure size 850x725 with 1 Axes>, 'axes': (<Axes: title={'center': 'Computed long ($L$)|short($S$) firing rate indicies'}, xlabel='Replay Firing Rate Index $\\frac{L_{R}-S_{R}}{L_{R} + S_{R}}$', ylabel='Laps Firing Rate Index $\\frac{L_{\\theta}-S_{\\theta}}{L_{\\theta} + S_{\\theta}}$'>,), 'plot_data': {}})

In [ ]:
import matplotlib
# configure backend here
# matplotlib.use('Qt5Agg')
# backend_qt5agg
matplotlib.use('Qt5Agg') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _plot_long_short_firing_rate_indicies


# Plot long|short firing rate index:
long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
x_frs_index, y_frs_index = long_short_fr_indicies_analysis_results['x_frs_index'], long_short_fr_indicies_analysis_results['y_frs_index'] # use the all_results_dict as the computed data value
active_context = long_short_fr_indicies_analysis_results['active_context']
fig, _temp_full_fig_save_path = _plot_long_short_firing_rate_indicies(x_frs_index, y_frs_index, active_context, fig_save_parent_path=None, debug_print=False)
fig.show()

In [132]:
ax = plt.gca()


In [133]:
ax.spines[['top', 'right']].set_visible(True)

In [ ]:
# curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_short_long_firing_rate_analyses'], fail_on_exception=True, debug_print=False) # fail_on_exception MUST be True or error handling is all messed up 
long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
x_frs_index, y_frs_index = long_short_fr_indicies_analysis_results['x_frs_index'], long_short_fr_indicies_analysis_results['y_frs_index'] # use the all_results_dict as the computed data value
active_context = long_short_fr_indicies_analysis_results['active_context']
long_short_fr_indicies_analysis_results

# Other Programmatic Figures

In [ ]:
batch_extended_programmatic_figures(curr_active_pipeline=curr_active_pipeline)

In [ ]:
batch_programmatic_figures(curr_active_pipeline=curr_active_pipeline)

In [ ]:
# Output the 2D placefields for my presentation
from neuropy.core.neuron_identities import PlotStringBrevityModeEnum
from neuropy.plotting.ratemaps import BackgroundRenderingOptions

In [ ]:

# '_display_2d_placefield_result_plot_ratemaps_2D'
# %matplotlib inline
%matplotlib qt
# curr_active_pipeline.display('_display_2d_placefield_result_plot_ratemaps_2D', long_epoch_context) # MatplotlibRenderPlots


In [ ]:

# long_pf2D.plot_occupancy()
_out = short_one_step_decoder_2D.pf.plot_ratemaps_2D(included_unit_neuron_IDs=[2,4,5], bg_rendering_mode=BackgroundRenderingOptions.EMPTY, use_special_overlayed_title=False, missing_aclu_string_formatter=None, debug_print=False, brev_mode=PlotStringBrevityModeEnum.NONE)

## Single column output: subplots=(None, 9)


In [ ]:
# %%capture
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_1d_placefields', debug_print=False) # 🟢✅ Now seems to be working and saving to PDF!! Still using matplotlib.use('Qt5Agg') mode and plots still appear.

In [ ]:
long_shared_aclus_only_decoder.pf.plot_occupancy()

In [ ]:
%matplotlib qt
long_one_step_decoder_2D.pf.plot_occupancy()

In [ ]:
long_one_step_decoder_2D.pf.plot_ratemaps_2D()

In [ ]:
# curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_session_configuration_context=global_epoch_context, single_figure=False)

short_one_step_decoder_1D

In [ ]:
%matplotlib inline
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

long_single_cell_pfmap_processing_fn = None
short_single_cell_pfmap_processing_fn = None

# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: 0.5 * pfmap # flip over the y-axis
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: -0.5 * pfmap # flip over the y-axis

# pad = 1
# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap) + (0.5*pad) # shift the baseline up by half
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (-0.5 * pfmap * pad) + (0.5*pad) # flip over the y-axis, shift the baseline down by half

# pad = 1
# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap * pad) + (0.5*pad) # shift the baseline up by half
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap * pad) + (0.5*pad) # flip over the y-axis, shift the baseline down by half
# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap * pad) # shift the baseline up by half
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap * pad) # flip over the y-axis, shift the baseline down by half


# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (1.0 * pfmap * pad) # shift the baseline up by half
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (-1.0 * pfmap * pad) + (1.0*pad) # this does not work and results in short being fully filled. I think this is because the fill_between gets reversed since everything is below baseline

sort_idx = None

out = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx, single_figure=False, debug_print=False, fignum='Short v Long pf1D Comparison',
                                   long_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
                                   short_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
                                  )

# ax = out.axes[0]

In [ ]:
sort_idx = np.argsort(product_overlap_scalars_df.prod_overlap.to_numpy())[::-1] # the `[::-1]` term reverses the array, which by defaul is returned in ascending order and we want descending
sort_idx

In [ ]:


curr_ratemap = long_one_step_decoder_1D.pf.ratemap
curr_ratemap.get_sort_indicies()
# .pf1D.ratemap

# 2023-04-27 - Idea: Candidate Replay "Quality" Metric
Observed that I visually distinguish "good" replays from bad ones based on their mostly-monotonically increasing nature.

#### They don't have to:
	Span the whole track
	Start or end at an end-cap
	Increase linearly

#### The algorithm must:
	tolerate ocasional jumps in an otherwise linear sequence
	allow sequences to "start" only halfway through. They should extract out the coherent sequence

#### Ideas:
	- just a linear fit
	- a monotonicity check using some sort of cumulative sum over the differences in position.
	- "radon transform"?

# 2023-05-02 - Session Validation Check Info
Generate info about the number of laps detected, the duration, the number of cells, the number of replays, etc for the active session so that it can be ensured that there wasn't an error that is messing up the analysis.

In [ ]:


curr_active_pipeline.active_sess_config

In [ ]:
from neuropy.utils.debug_helpers import debug_print_placefield, debug_print_spike_counts, debug_print_subsession_neuron_differences, print_aligned_columns
from neuropy.utils.misc import print_seconds_human_readable

debug_print_spike_counts(global_session)

In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _compare_computation_results
# curr_active_pipeline.computation_results
# pf_neurons_diff = _compare_computation_results(curr_active_pipeline.computation_results.maze1_PYR, curr_active_pipeline.computation_results.maze2_PYR)
pf_neurons_diff = _compare_computation_results(long_results, short_results)
pf_neurons_diff

In [ ]:
print(pf_neurons_diff)

In [ ]:

from neuropy.analyses.laps import estimate_session_laps

curr_laps = global_session.laps
curr_laps.to_dataframe()

In [ ]:

# Build the new laps first:
# global_session = estimate_session_laps(global_session)

global_session.replace_session_laps_with_estimates()

### Plotting Validations

In [ ]:
%matplotlib qt
short_one_step_decoder_1D.pf.plot_ratemaps_1D()

In [ ]:
short_results.pf1D_Decoder.pf.plot_ratemaps_1D()

In [ ]:
short_results.pf2D_Decoder.pf.plot_ratemaps_2D()
short_one_step_decoder_2D.pf.plot_ratemaps_2D()

In [ ]:
%matplotlib qt
long_session.plot_laps_2d()

In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_laps_2d

%matplotlib qt
fig, out_axes_list = plot_laps_2d(global_session, legacy_plotting_mode=False)
out_axes_list[0].set_title('New Pho Position Thresholding Estimated Laps')
fig.canvas.manager.set_window_title('New Pho Position Thresholding Estimated Laps')

In [ ]:
## Laps 
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.Mixins.LapsVisualizationMixin import LapsVisualizationMixin

curr_position_df, lap_specific_position_dfs = LapsVisualizationMixin._compute_laps_specific_position_dfs(curr_active_pipeline.sess)
lap_specific_position_dfs = [curr_position_df.groupby('lap').get_group(i)[['t','x','y','lin_pos']] for i in curr_active_pipeline.sess.laps.lap_id] # dataframes split for each ID:
laps_position_times_list = [np.squeeze(lap_pos_df[['t']].to_numpy()) for lap_pos_df in lap_specific_position_dfs]
laps_position_traces_list = [lap_pos_df[['x','y']].to_numpy().T for lap_pos_df in lap_specific_position_dfs]
## Build Epochs:
epochs = curr_active_pipeline.sess.laps.to_dataframe()
epoch_slices = epochs[['start', 'stop']].to_numpy()
epoch_description_list = [f'lap {epoch_tuple.lap_id} (maze: {epoch_tuple.maze_id}, direction: {epoch_tuple.lap_dir})' for epoch_tuple in epochs[['lap_id','maze_id','lap_dir']].itertuples()]
# print(f'epoch_description_list: {epoch_description_list}') # epoch_descriptions: ['lap 41 (maze: 2, direction: 1)', 'lap 42 (maze: 2, direction: 0)', 'lap 43 (maze: 2, direction: 1)', 'lap 44 (maze: 2, direction: 0)', 'lap 45 (maze: 2, direction: 1)', 'lap 46 (maze: 2, direction: 0)', 'lap 47 (maze: 2, direction: 1)', 'lap 48 (maze: 2, direction: 0)', 'lap 49 (maze: 2, direction: 1)', 'lap 50 (maze: 2, direction: 0)', 'lap 51 (maze: 2, direction: 1)', 'lap 52 (maze: 2, direction: 0)', 'lap 53 (maze: 2, direction: 1)', 'lap 54 (maze: 2, direction: 0)', 'lap 55 (maze: 2, direction: 1)', 'lap 56 (maze: 2, direction: 0)', 'lap 57 (maze: 2, direction: 1)', 'lap 58 (maze: 2, direction: 0)', 'lap 59 (maze: 2, direction: 1)', 'lap 60 (maze: 2, direction: 0)', 'lap 61 (maze: 2, direction: 1)', 'lap 62 (maze: 2, direction: 0)', 'lap 63 (maze: 2, direction: 1)', 'lap 64 (maze: 2, direction: 0)', 'lap 65 (maze: 2, direction: 1)', 'lap 66 (maze: 2, direction: 0)', 'lap 67 (maze: 2, direction: 1)', 'lap 68 (maze: 2, direction: 0)', 'lap 69 (maze: 2, direction: 1)', 'lap 70 (maze: 2, direction: 0)', 'lap 71 (maze: 2, direction: 1)', 'lap 72 (maze: 2, direction: 0)', 'lap 73 (maze: 2, direction: 1)', 'lap 74 (maze: 2, direction: 0)', 'lap 75 (maze: 2, direction: 1)', 'lap 76 (maze: 2, direction: 0)', 'lap 77 (maze: 2, direction: 1)', 'lap 78 (maze: 2, direction: 0)', 'lap 79 (maze: 2, direction: 1)']
# -

In [ ]:

def debug_print_subsession_epochs_differences(num_updated_total_items: int, num_original_total_items: int, item_label=None, subsession_name=None):
    print('{}/{} total {} remain in subsession {}'.format(num_updated_total_items, num_original_total_items, (item_label or "items"), (subsession_name or "")))


debug_print_subsession_epochs_differences(num_updated_total_items=len(long_session.replay), num_original_total_items=len(long_session.replay_backup), item_label='replays', subsession_name='"long"')
debug_print_subsession_epochs_differences(num_updated_total_items=len(short_session.replay), num_original_total_items=len(short_session.replay_backup), item_label='replays', subsession_name='"short"')

print_aligned_columns
# 